In [ ]:
%matplotlib inline
import os
os.environ['PY3_PROD'] = '1'
%load_ext autoreload
%autoreload 2
os.system('kinit')

In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib
from pycmqlib3.utility import dbaccess, dataseries, misc
from pycmqlib3.analytics.tstool import *
from pycmqlib3.analytics.btmetrics import *
from pycmqlib3.analytics.backtest_utils import *

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
matplotlib.rcParams['figure.figsize'] = (12, 8)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

lead_lag_config = {
    'll_left': -20,
    'll_right': 120,
    'll_spacing': 5,
    'll_sub_win': [(datetime.date(2008, 1, 1), datetime.date(2016, 12, 31)), 
                   (datetime.date(2017, 1, 1), datetime.date(2023, 12, 31)),],
}

ll_keys = ['fullsample'] + ['%s:%s' % (sd.strftime('%Y-%b-%d'), ed.strftime('%Y-%b-%d')) for sd, ed in lead_lag_config['ll_sub_win']]


# load historical data

In [ ]:
from misc_scripts.update_fut_prices import load_saved_fut
tday = datetime.date(2025, 1, 3)

df = load_saved_fut(tday, freq='d')
#df = load_cnc_fut(tday, type='cnc')

spot_df = load_fun_data(tday)

start_date = df.index[0]
end_date = tday

cdates = pd.date_range(start=start_date, end=tday, freq='D')
bdates = pd.bdate_range(start=start_date, end=end_date, freq='C', holidays=misc.CHN_Holidays)

# Calculate features

In [ ]:
mr_dict = {
    'hc': 'rb',
    'i': 'rb',
    'j': 'rb',
#     'v': 'pp',
    'pp': 'l',
    'SA': 'FG',
    'sc': 'lu',
    'lu': 'fu',   
    'm': 'y',
#     'p': 'y',
#     'SM': 'SF',

}

leadlag_dict = {     
    #'jm': 'j',
    'lu': 'sc',    
    'RM': 'm',
#     'y': 'p',
#     'OI': 'p',
    'cs': 'c',
    'eb': 'sc',
#     'fu': 'sc',
    'bu': 'sc',
#     'PF': 'eg',
    'bc': 'cu',
    'MA': 'pp',    
    'ni': 'ss',
    'al': 'ao',  
    'ag': 'au',
    'TF': 'T',
    'IF': 'IC'        
}


In [ ]:
vol_win=20

pnl_tenors = ['6m', '1y', '2y', '3y', '4y', '5y', '6y', '7y', '8y', '9y']

empiric_assets = [
    'rb', 'hc', 'i', 'j', 'jm', 
    'SM', 'SF',
    'l', 'pp', 'v', 'lu', 'sc',     
    'TA', 'PF', 'eg', 'MA', 'eb', 'fu', 'bu',
    'zn', 
    'FG', 'SA', 
    'ss', 'ni', 
    'cu', 'bc', 
    'c', 'cs', 
    'ru', 'nr',
    'ao', 'al',
    'm', 'RM', 'y', 'OI', 'p', 'a', 'b', 
    'TF', 'T', 'au', 'ag', 'IF', 'IC', 'IH',
    
]

cont = 'c1'
px_lbl = 'a1535'

asset_conts = [asset + 'c1' for asset in empiric_assets]
adf = df.loc[:, df.columns.get_level_values(0).isin(asset_conts)].copy()

for asset_cont in asset_conts:
    for (n_lbl, d_lbl) in [('n305', 'a1505'), ('n310', 'a1510'), ('n450', 'p2055')]:
        flag = adf[(asset_cont, n_lbl)].isna()
        adf.loc[flag, (asset_cont, n_lbl)] = adf.loc[flag, (asset_cont, d_lbl)]

#flag = (adf[('cuc1', 'n305')].isna()) & (~adf[('cuc1', 'a1505')].isna())
# for asset_cont in asset_conts:
#     adf.loc[flag, (asset_cont, px_lbl)] = adf.loc[flag, (asset_cont, 'a1505')]

df_pxchg = pd.DataFrame(index=adf.index, columns=empiric_assets)

for asset in empiric_assets:
    if '_' in asset:
        df_pxchg[asset] = beta_ret_dict[asset].dropna()
    else:
        df_pxchg[asset] = adf[(f"{asset}{cont}", px_lbl)].dropna().pct_change()

vol_df = df_pxchg.rolling(vol_win).std()

In [ ]:
cutoff='2010-01-01'
#feature='lme_futbasis'
signal_cap= [-2, 2]
vol_win = 60
param_rng = [1, 2, 1]
signal_func = 'ma'

bullish = False
#freq='W-Fri'

win = 10
signal_df = pd.DataFrame(index=bdates, columns=empiric_assets)

for asset in mr_dict:
    #feature_ts = spot_df[asset + '_' + feature].dropna() #reindex(index=cdates).ffill().reindex(index=bdates)    
    tmp_df = df[[(mr_dict[asset]+'c1', 'close'), (asset+'c1', 'close')]].dropna(how='all').ffill().droplevel([1], axis=1)
    tmp_df = tmp_df.pct_change()    
    tmp_df = tmp_df.rolling(win).sum()
    feature_ts = tmp_df[asset+'c1'] - tmp_df[mr_dict[asset]+'c1']     
    signal_ts = calc_conv_signal(feature_ts, signal_func=signal_func, param_rng=param_rng, signal_cap=signal_cap, vol_win=vol_win)
    signal_ts = signal_ts.ewm(3).mean()
    #signal_ts = signal_buffer(signal_ts, 0.8)
    signal_ts = signal_ts.reindex(index=cdates).ffill().reindex(index=df_pxchg.index).dropna()
    signal_df[asset] = signal_ts
    
signal_df2 = pd.DataFrame(index=bdates, columns=empiric_assets)
for asset in leadlag_dict:
    #feature_ts = spot_df[asset + '_' + feature].dropna() #reindex(index=cdates).ffill().reindex(index=bdates)    
    tmp_df = df[[(leadlag_dict[asset]+'c1', 'close'), (asset+'c1', 'close')]].dropna(how='all').ffill().droplevel([1], axis=1)
    tmp_df = tmp_df.pct_change()    
    tmp_df = tmp_df.rolling(win).sum()
    signal_ts = tmp_df[leadlag_dict[asset]+'c1'] - tmp_df[asset+'c1']        
    signal_ts = calc_conv_signal(signal_ts, signal_func=signal_func, param_rng=param_rng, signal_cap=signal_cap, vol_win=vol_win)    
    signal_ts = signal_ts.ewm(3).mean()
    signal_ts = signal_ts.reindex(index=cdates).ffill().reindex(index=df_pxchg.index).dropna()
    signal_df2[asset] = signal_ts

signal_df = signal_df.fillna(0) + signal_df2.fillna(0) 
#signal_df = xs_demean(signal_df) 
#signal_df = signal_buffer(signal_df, 0.1)

#signal_df = xs_demean(signal_df)
holding = generate_holding_from_signal(signal_df, vol_df, risk_scaling=1.0, asset_scaling=False)

bt_metrics = MetricsBase(holdings=holding[empiric_assets][cutoff:],
                         returns=df_pxchg[empiric_assets][cutoff:], 
                         shift_holdings=1)

bt_metrics_w_cost = MetricsBase(holdings=holding[empiric_assets][cutoff:],
                                returns=df_pxchg[empiric_assets][cutoff:], 
                                shift_holdings=1,
                                cost_dict=simple_cost(holding.columns, trd_cost=3e-4))

pnl_stats = bt_metrics.calculate_pnl_stats(shift=0, use_log_returns=False, tenors=pnl_tenors, perf_metrics=['sharpe', 'std', 'sortino', 'calmar'])
pnl_stats_w_cost = bt_metrics_w_cost.calculate_pnl_stats(shift=0, use_log_returns=False, tenors=pnl_tenors, perf_metrics=['sharpe', 'std', 'sortino', 'calmar'])

print("SR after cost:\n", pnl_stats_w_cost['sharpe'])
print(pnl_stats_w_cost['asset_sharpe_stats'])
print("Turnover: \n%s\nPNL per trade:\n%s\n" % (pnl_stats_w_cost['turnover'], pnl_stats_w_cost['pnl_per_trade']))

print("SR before cost:\n", pnl_stats['sharpe'])
print(pnl_stats['asset_sharpe_stats'])
print("Turnover: \n%s\nPNL per trade:\n%s\n" % (pnl_stats['turnover'], pnl_stats['pnl_per_trade']))

iplot(pnl_stats_w_cost['portfolio_cumpnl'], title='portfolio pnl with cost')
iplot(pnl_stats_w_cost['asset_cumpnl'], title='asset pnl with cost')

iplot(pnl_stats['portfolio_cumpnl'], title='portfolio pnl wo cost')
iplot(pnl_stats['asset_cumpnl'], title='asset pnl wo cost')


In [ ]:
pnl_stats_w_cost['std']

In [ ]:
lead_lag_config = {
    'll_left': -20,
    'll_right': 120,
    'll_spacing': 5,
    'll_sub_win': [(datetime.date(2008, 1, 1), datetime.date(2018, 12, 31)), 
                   (datetime.date(2019, 1, 1), datetime.date(2024, 12, 31)),],
}

ll_keys = ['fullsample'] + ['%s:%s' % (sd.strftime('%Y-%b-%d'), ed.strftime('%Y-%b-%d')) for sd, ed in lead_lag_config['ll_sub_win']]


ll_left = lead_lag_config['ll_left']
ll_right = lead_lag_config['ll_right']
spacing = lead_lag_config['ll_spacing']

leadlag_df = bt_metrics.lead_lag(ll_limit_left=ll_left, 
                                 ll_limit_right=ll_right,
                                 ll_sub_windows=lead_lag_config['ll_sub_win'])

fig, ax = plt.subplots(len(ll_keys), 1)
fig.set_figheight(15)
fig.set_figwidth(10)

for i, key in enumerate(ll_keys):
    ts = leadlag_df['leadlag_sharpes'].loc[key]
    ts.plot(kind='bar', ax = ax[i], title = f'lead_lag: {key}')
    new_ticks = np.linspace(ll_left, ll_right, (ll_right-ll_left)//spacing+1)
    ax[i].set_xticks(np.interp(new_ticks, ts.index, np.arange(ts.size)))
    ax[i].set_xticklabels(new_ticks)
    ax[i].axvline(x=-ll_left, color='red', linestyle='--')
plt.show()

# fig = plt.figure()
# ax = fig.add_subplot(111)
# ls_pnl = bt_metrics.long_short_pnl()
# for key in ls_pnl:
#     ax.plot(ls_pnl[key]['portfolio_cumpnl'].index, ls_pnl[key]['portfolio_cumpnl'].values, '-', label=key)
# lines, labels = ax.get_legend_handles_labels()
# ax.legend(lines, labels, bbox_to_anchor=(1.04, 1), loc='upper left')
# ax.grid()
# plt.title("long-short pnl")
# plt.show()

lagged = bt_metrics.lagged_pnl(lags=[1, 5, 10, 20, 30, 60, 75, 80])
lagged['cumpnl'].plot()
#print('lagged PNL\n', lagged['sharpe'])
plt.grid()
plt.title('lagged pnl')
plt.show()

smoothed = bt_metrics.smoothed_pnl(smooth_hls=[1, 5, 10, 20, 30, 60, 75, 80])
smoothed['cumpnl'].plot(figsize=(8, 6))
#print('smoothed PNL\n', smoothed['sharpe'])
plt.grid()
plt.title('smoothed pnl')
plt.show()

#tilt_timing = bt_metrics.tilt_timing(tilt_rolling_window=1*244) # default 3 years  tilt_rolling_window = 3 * 244 

seasonal_pnl = bt_metrics.seasonal_pnl()
cumpnl = seasonal_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('seasonal sharpe stats\n', seasonal_pnl['sharpe_stats'])
plt.grid()
plt.title('monthly pnl')
plt.show()


monthday_pnl = bt_metrics.monthday_pnl()
cumpnl = monthday_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('monthday sharpe stats\n', monthday_pnl['sharpe_stats'])
plt.grid()
plt.title('monthday pnl')
plt.show()


week_pnl = bt_metrics.week_pnl()
cumpnl = week_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('week sharpe stats\n', week_pnl['sharpe_stats'])
plt.grid()
plt.title('weekday pnl')
plt.show()


annual_pnl = bt_metrics.annual_pnl()
cumpnl = annual_pnl['cumlog_pnl']
cumpnl.set_index(cumpnl.index.astype('str')).plot(rot=30, figsize = (8, 6))
#print('annual sharpe stats\n', annual_pnl['sharpe_stats'])
plt.grid()
plt.title('annual pnl')
plt.show()

annual_pnl['cumlog_pnl'].mean(axis=1).plot()
plt.grid()
plt.title('annual averaged profile')
plt.show()

# turnover = bt_metrics.turnover()
# print(turnover)